In [1]:
import mpp_classifiers as mpp
import load_ship_data as lsd
import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold

MPP notebook

get data via our data loader

In [2]:
np.random.seed(522)
data_path="data/shipsnet.json"
train,test,valid=lsd.load_data_train_test_split(data_path)
def conf(pred,y):
    T0=sum([1 if x==y and y==0 else 0 for (x,y) in zip(pred,y)])
    T1=sum([1 if x==y and y==1 else 0 for (x,y) in zip(pred,y)])
    F0=sum([1 if x!=y and y==0 else 0 for (x,y) in zip(pred,y)])
    F1=sum([1 if x!=y and y==1 else 0 for (x,y) in zip(pred,y)])
    return np.array([[T0,F0],[F1,T1]])

Training Set Data Length:  2800   Label Length:  2800
TestingSet Set Data Length:  600  Label Length:  600
Validation Set Data Length:  600  Label Length:  600


Setup data splits and priors

Note: we split into train/valid/test as above, then we merge train+valid

This arrangement is due to scikit learn handling validation for us, a feature we were unaware of when designing data loader

In [3]:
ship_prob=np.sum(train[1])*1.0/train[1].shape[0]
p=np.array([1-ship_prob,ship_prob])
print(p)
Xtrain=train[0]
ytrain=train[1]
Xtest=test[0]
ytest=test[1]
Xvalid=valid[0]
yvalid=valid[1]
Xtrain=Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1]*Xtrain.shape[2]*Xtrain.shape[3])
Xtest=Xtest.reshape(Xtest.shape[0],Xtest.shape[1]*Xtest.shape[2]*Xtest.shape[3])
Xvalid=Xvalid.reshape(Xvalid.shape[0],Xvalid.shape[1]*Xvalid.shape[2]*Xvalid.shape[3])
print(Xtrain.shape)
print(Xtest.shape)
print(Xvalid.shape)
Xtrain=np.concatenate((Xtrain,Xvalid))
ytrain=np.concatenate((ytrain,yvalid))
print(Xtrain.shape)
print(ytrain.shape)

[0.7525 0.2475]
(2800, 19200)
(600, 19200)
(600, 19200)
(3400, 19200)
(3400,)


Case 1. No PCA, even priors

In [27]:

p55=np.array([.5,.5])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p55)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print("Confusion Matrix:")
print(ar)

Overall accuracy = 0.6766666666666666;
Classwise accuracy = [0.68666667 0.64666667];
The learning process takes 31.545286417007446 seconds.
Confusion Matrix:
[[309 141]
 [ 53  97]]


Case 2. No PCA, prior assumes ships are rare

In [26]:
p91=np.array([.9,.1])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p91)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print("Confusion Matrix:")
print(ar)

Overall accuracy = 0.75;
Classwise accuracy = [1. 0.];
The learning process takes 33.399874210357666 seconds.
Confusion Matrix:
[[450   0]
 [150   0]]


Case 1. No PCA, priors from training data

In [25]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print("Confusion Matrix:")
print(ar)

Overall accuracy = 0.75;
Classwise accuracy = [1. 0.];
The learning process takes 32.428704023361206 seconds.
Confusion Matrix:
[[450   0]
 [150   0]]


PCA = 850 dimensions kept

In [7]:
pca = PCA(n_components=850)
pca.fit(Xtrain)
pca_Xtrain=pca.transform(Xtrain)
pca_Xtest=pca.transform(Xtest)
explained=np.sum(pca.explained_variance_ratio_)
print(explained)

0.9901861232274156


Case 1. PCA=850, training priors used from now on

In [28]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print("Confusion Matrix:")
print(ar)

Overall accuracy = 0.75;
Classwise accuracy = [1. 0.];
The learning process takes 0.14661884307861328 seconds.
Confusion Matrix:
[[450   0]
 [150   0]]


Case 2. PCA=850

In [29]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print("Confusion Matrix:")
print(ar)

Overall accuracy = 0.9066666666666666;
Classwise accuracy = [0.93333333 0.82666667];
The learning process takes 58.8439154624939 seconds.
Confusion Matrix:
[[420  30]
 [ 26 124]]


Case 3. PCA=850

In [30]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 3, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print("Confusion Matrix:")
print(ar)

Overall accuracy = 0.75;
Classwise accuracy = [1. 0.];
The learning process takes 83.75339984893799 seconds.
Confusion Matrix:
[[450   0]
 [150   0]]


PCA =245 dimensions kept

In [4]:
pca2 = PCA(n_components=245)
pca2.fit(Xtrain)
pca2_Xtrain=pca2.transform(Xtrain)
pca2_Xtest=pca2.transform(Xtest)
explained=np.sum(pca2.explained_variance_ratio_)
print(explained)

0.9497604252286876


Case 1. PCA=245

In [31]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print("Confusion Matrix:")
print(ar)

Overall accuracy = 0.75;
Classwise accuracy = [1. 0.];
The learning process takes 0.047041893005371094 seconds.
Confusion Matrix:
[[450   0]
 [150   0]]


Case 2. PCA=245

In [7]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print("Confusion Matrix:")
print(ar)
#np.save("mppcase2_test.npy",y_model)
#np.save("truth_train.npy",ytrain)
#np.save("truth_test.npy",ytest)
#y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtrain, 2, p)
#acc_classwise, acc_overall = mpp.accuracy_score(ytrain, y_model)
#print(acc_overall)
#np.save("mppcase2_train.npy",y_model)

Overall accuracy = 0.9133333333333333;
Classwise accuracy = [0.95111111 0.8       ];
The learning process takes 0.745518684387207 seconds.
Confusion Matrix:
[[428  22]
 [ 30 120]]


Case 3. PCA=245

In [33]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 3, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print("Confusion Matrix:")
print(ar)

Overall accuracy = 0.785;
Classwise accuracy = [0.78 0.8 ];
The learning process takes 3.4766292572021484 seconds.
Confusion Matrix:
[[351  99]
 [ 30 120]]


Setup for Kfold cross-validation

In [15]:
p=np.array([1-ship_prob,ship_prob])
X=pca2_Xtrain
y=ytrain
k=5

Case 1

In [42]:
kf=StratifiedKFold(n_splits=k,random_state=None)
acc_scores=[]
conf_mat=[]
times=[]
for train_index,test_index in kf.split(X,y):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    t0=time.time()
    y_model = mpp.mpp(X_train, y_train, X_test, 1, p)
    t1=time.time()
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    t=t1-t0
    times.append(t)
    acc_scores.append(acc_overall)
    conf_mat.append(conf(y_model,y_test))
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
print('Average Runtime = {}'.format(np.sum(np.array(times))*1.0/5))
class0_acc=np.array([x[0][0]*1.0/(x[0][0]+x[0][1]) for x in conf_mat])
class0_acc=np.sum(class0_acc)*1.0/5
class1_acc=np.array([x[1][1]*1.0/(x[1][0]+x[1][1]) for x in conf_mat])
class1_acc=np.sum(class1_acc)*1.0/5
print(class0_acc)
print(class1_acc)

Accuracy from each fold =  [0.7514705882352941, 0.7514705882352941, 0.7514705882352941, 0.7514705882352941, 0.7529411764705882]
Average Accuracy = 0.7517647058823529
Average Runtime = 0.05354189872741699
1.0
0.007058823529411765


Case 2

In [43]:
acc_scores=[]
conf_mat=[]
times=[]
for train_index,test_index in kf.split(X,y):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    t0=time.time()
    y_model = mpp.mpp(X_train, y_train, X_test, 2, p)
    t1=time.time()
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    t=t1-t0
    times.append(t)
    acc_scores.append(acc_overall)
    conf_mat.append(conf(y_model,y_test))
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
print('Average Runtime = {}'.format(np.sum(np.array(times))*1.0/5))
class0_acc=np.array([x[0][0]*1.0/(x[0][0]+x[0][1]) for x in conf_mat])
class0_acc=np.sum(class0_acc)*1.0/5
class1_acc=np.array([x[1][1]*1.0/(x[1][0]+x[1][1]) for x in conf_mat])
class1_acc=np.sum(class1_acc)*1.0/5
print(class0_acc)
print(class1_acc)

Accuracy from each fold =  [0.9176470588235294, 0.9191176470588235, 0.9220588235294118, 0.9147058823529411, 0.9352941176470588]
Average Accuracy = 0.9217647058823528
Average Runtime = 2.2008477210998536
0.9572549019607843
0.8152941176470587


Case 3

In [44]:
acc_scores=[]
conf_mat=[]
times=[]
for train_index,test_index in kf.split(X,y):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    t0=time.time()
    y_model = mpp.mpp(X_train, y_train, X_test, 3, p)
    t1=time.time()
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    t=t1-t0
    times.append(t)
    acc_scores.append(acc_overall)
    conf_mat.append(conf(y_model,y_test))
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
print('Average Runtime = {}'.format(np.sum(np.array(times))*1.0/5))
class0_acc=np.array([x[0][0]*1.0/(x[0][0]+x[0][1]) for x in conf_mat])
class0_acc=np.sum(class0_acc)*1.0/5
class1_acc=np.array([x[1][1]*1.0/(x[1][0]+x[1][1]) for x in conf_mat])
class1_acc=np.sum(class1_acc)*1.0/5
print(class0_acc)
print(class1_acc)

Accuracy from each fold =  [0.8088235294117647, 0.8044117647058824, 0.7661764705882353, 0.7911764705882353, 0.8044117647058824]
Average Accuracy = 0.7949999999999999
Average Runtime = 3.600688362121582
0.8152941176470587
0.7341176470588235


In [40]:
pca3 = PCA(n_components=50)
pca3.fit(Xtrain)
pca3_Xtrain=pca3.transform(Xtrain)
pca3_Xtest=pca3.transform(Xtest)
explained=np.sum(pca3.explained_variance_ratio_)
print("Variance explained: ", explained)
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca3_Xtrain, ytrain, pca3_Xtest, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print("Confusion Matrix:")
print(ar)
#np.save("ground_truth.npy",ytrain)
#np.save("mpp_case2.npy",y_model)

Variance explained:  0.8469294363086588
Overall accuracy = 0.8916666666666667;
Classwise accuracy = [0.93333333 0.76666667];
The learning process takes 0.22038793563842773 seconds.
Confusion Matrix:
[[420  30]
 [ 35 115]]
